In [1]:
#필요한 라이브러리 import
import requests
import urllib.request
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pymysql

In [34]:
conn = pymysql.connect(host="localhost", user='queant', password='queant1234', db = 'queant', charset='utf8mb4')
cur = conn.cursor()

**의논사항**
* product table에서 은행 id Long으로 선언되어 있었는데 Long이 없는것 같아서 int로 변경
* 가입방법 table에 가입방법 column이 필요 없을 것 같다고 생각이 듦.

create product table
```sql
CREATE TABLE `queant`.`product` (
  `product_id` INT NOT NULL AUTO_INCREMENT,
  `bank_id` INT NOT NULL,
  `is_deposit` TINYINT NOT NULL,
  `name` VARCHAR(50) NOT NULL,
  PRIMARY KEY (`product_id`));
```

create joinway table
```sql
CREATE TABLE `queant`.`joinway` (
  `joinway_id` INT NOT NULL AUTO_INCREMENT,
  `product_id` INT NOT NULL,
  `code_id` VARCHAR(2) NOT NULL,
  `scode_id` INT NOT NULL,
  PRIMARY KEY (`joinway_id`));
```


In [3]:
#api활용할 url부분

#각 데이터 별 url 앞부분
url_deposit = ["https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=", "https://finlife.fss.or.kr/finlifeapi/savingProductsSearch.xml?auth="]

#발급받은 키
api_key = "47c0e868fdb16333d47d0e385641c3c0"

#권역코드
code_front = "&topFinGrpNo="
code_bank = ["020000", "030300"]

#페이지
page_front = "&pageNo="
page_num = ["1", "2", "3", "4"]

In [4]:
is_deposit = 1
url = url_deposit[0] + api_key + code_front + code_bank[is_deposit] + page_front + page_num[0]
print(url)
data_str = urllib.request.urlopen(url).read().decode('euc-kr')
data_xml = ET.fromstring(data_str)

https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=47c0e868fdb16333d47d0e385641c3c0&topFinGrpNo=030300&pageNo=1


In [ ]:
#상품 table에 정보들을 담는다.
#python은 전부다 %s를 써야함.
query_prdt = """INSERT INTO queant.product (bank_id, is_deposit, name) values (%s,%s,%s);"""
for product_tag in data_xml[5]:
    bank_id = int(product_tag[0].find("fin_co_no").text) #은행코드
    deposit = is_deposit
    prdt_name = product_tag[0].find("fin_prdt_nm").text #상품명
    prdt_name = prdt_name.replace("\n", " ")
    values = (bank_id,deposit,prdt_name) # 각각의 value들을 values에 묶어서 전달한다.
    cur.execute(query_prdt, values)
conn.commit()

In [27]:
# 공통코드에서 가입방법의 코드를 가져온다.
cur.execute("SELECT code_id FROM queant.commoncode where code_value = \"가입 방법\"")

row = cur.fetchone()
if row != None:
    code_id = row[0] #A가 저장됨

In [30]:
#가입 방법 코드
cur.execute("SELECT * FROM queant.specificcode where code_id = \'A\'")

join_scode_id = {}
while True:
    row = cur.fetchone()
    if row == None:
        break
    join_scode_id[row[2]] = int(row[0])
    
print(join_scode_id["전화"])

3


In [36]:
query_join = """INSERT INTO queant.joinway (product_id, code_id, scode_id) values (%s,%s,%s);"""
query_find_prdt = """select product_id from queant.product where name = (%s) and bank_id = (%s)"""
for product_tag in data_xml[5]:
    prdt_name = product_tag[0].find("fin_prdt_nm").text #상품명
    bank_id = int(product_tag[0].find("fin_co_no").text) #은행코드
    values = (prdt_name, bank_id)
    cur.execute(query_find_prdt, values)
    product_id = cur.fetchone()[0]
    join = product_tag[0].find("join_way").text
    join_ways = ["영업점", "인터넷", "스마트폰", "전화", "기타"]
    for join_way in join_ways:
        if join_way in join:
            values = (product_id, code_id, join_scode_id[join_way])
            cur.execute(query_join, values)
conn.commit()
            
    

In [38]:
cur.execute("select * from queant.joinway as jw  left join queant.product as prdt on prdt.product_id = jw.product_id right join queant.specificcode as spec on jw.scode_id = spec.scode_id;")
while True:
    row = cur.fetchone()
    if row == None:
        break
    print(row)


(159, 1, 'A', 0, 1, 10345, 1, '정기예금', 0, 'A', '영업점')
(163, 2, 'A', 0, 2, 10345, 1, '플러스회전식정기예금(변동금리)', 0, 'A', '영업점')
(166, 4, 'A', 0, 4, 10345, 1, '3-UP 정기예금', 0, 'A', '영업점')
(170, 6, 'A', 0, 6, 10346, 1, '정기예금', 0, 'A', '영업점')
(171, 7, 'A', 0, 7, 10346, 1, 'CD연동장기정기예금', 0, 'A', '영업점')
(176, 7, 'A', 0, 7, 10346, 1, 'CD연동장기정기예금', 0, 'A', '영업점')
(179, 10, 'A', 0, 10, 10346, 1, 'OSB회전식정기예금', 0, 'A', '영업점')
(183, 12, 'A', 0, 12, 10349, 1, 'Dream Big 정기예금 (변동금리)', 0, 'A', '영업점')
(187, 14, 'A', 0, 14, 10349, 1, '정기예금', 0, 'A', '영업점')
(192, 17, 'A', 0, 17, 10350, 1, '정기예금', 0, 'A', '영업점')
(199, 20, 'A', 0, 20, 10354, 1, '정기예금', 0, 'A', '영업점')
(203, 24, 'A', 0, 24, 10356, 1, '푸른 정기예금', 0, 'A', '영업점')
(206, 25, 'A', 0, 25, 10358, 1, '정기예금', 0, 'A', '영업점')
(209, 28, 'A', 0, 28, 10358, 1, '회전정기예금', 0, 'A', '영업점')
(212, 31, 'A', 0, 31, 10359, 1, '정기예금', 0, 'A', '영업점')
(216, 34, 'A', 0, 34, 10359, 1, '회전yes정기예금(1년단위 변동금리상품)', 0, 'A', '영업점')
(220, 37, 'A', 0, 37, 10363, 1, '정기예금', 0, 'A', '영업점')
(2

In [39]:
conn.close()